## Image Classification on Fashion MNIST Dataset using Convolutional Neural Network


<p align="center">
  <img src="https://storage.googleapis.com/kaggle-datasets-images/2243/3791/9384af51de8baa77f6320901f53bd26b/dataset-cover.png" />
  Image source: https://www.kaggle.com/
</p>

- Fashion training set consists of 70,000 images divided into 60,000 training and 10,000 testing samples. Dataset sample consists of 28x28 grayscale image, associated with a label from 10 classes.
<br><br>
- The 10 classes are as follows:
  - 0 => T-shirt/top
  - 1 => Trouser
  - 2 => Pullover
  - 3 => Dress
  - 4 => Coat
  - 5 => Sandal
  - 6 => Shirt
  - 7 => Sneaker
  - 8 => Bag
  - 9 => Ankle boot
<br><br>
- Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255.

## Library Imports

In [14]:
import numpy as np
import tensorflow as tf

## Load the Dataset and Normalize

In [15]:
mnist = tf.keras.datasets.fashion_mnist

# Load the training and test images
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

In [16]:
# In order to pass it on to convolution layer, we need to resize the images
training_images = training_images.reshape(len(training_images), 28, 28, 1)
test_images = test_images.reshape(len(test_images), 28, 28, 1)

In [17]:
# Normalize the images
training_images = training_images / 255.0
test_images = test_images / 255.0

## Build the Neural Network

In [18]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation=tf.nn.relu, input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [19]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
# Check the summary of the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               204928    
_________________________________________________________________
dense_3 (Dense)              (None, 10)               

In [21]:
# Training will stop, when specific threshold is reached

class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epochs, logs=None):
        if logs is None:
            logs = {}
        if logs.get('accuracy') > 0.92:
            print("Reached threshold accuracy level and hence cancelling training")
            self.model.stop_training = True

## Training the Model

In [22]:
callbacks = MyCallback()
model.fit(training_images, training_labels, epochs = 10, callbacks=[callbacks])

Epoch 1/10
1875/1875 [==============================] - 54s 29ms/step - loss: 0.5889 - accuracy: 0.7865
Epoch 2/10
1875/1875 [==============================] - 55s 29ms/step - loss: 0.2985 - accuracy: 0.8920
Epoch 3/10
1875/1875 [==============================] - 59s 31ms/step - loss: 0.2473 - accuracy: 0.9096
Epoch 4/10
1875/1875 [==============================] - 60s 32ms/step - loss: 0.2076 - accuracy: 0.9218
Reached threshold accuracy level and hence cancelling training


In [23]:
# evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Overall Accuracy on the test images - {test_acc}")

313/313 [==============================] - 1s 4ms/step - loss: 0.2593 - accuracy: 0.9070
Overall Accuracy on the test images - 0.9070000052452087
